# Using Case Objects<a class="anchor" id="using-case"></a>
Here, we will highlight some of the usefule features of case objects, starting from restoring and plotting data, to running parameter scans.

## TOC:
* [Restoring a UEDGE case](#restoring)
    * [From an input file](#inputfile)
    * [From a save file](#savefile)
    * [From memory](#memory)
    * [As a static Case](#static)
    * [Case options](#caseoptions)
* [Getting information about UEDGE using UETOOLS](#information)
    * [Information about setup](#aboutsetup)
    * [Search variable by name](#searchvarname)
    * [Search variable description](#search)
    * [Get variable information](#about)
    * [Access variable descriptors](#descriptors)
* [Managing UEDGE data using UETOOLS](#managing-data)
    * [Case.populate](#populate)
    * [Case.get](#get)
    * [Case.setue](#setue)
    * [Case.assign](#assign)
* [Saving and restoring](#save_restore)
    * [Case.restore_save](#restore)
    * [Case.save](#save)

## Restoring a UEDGE case<a class="anchor" id="restoring"></a>
The first thing to do, is to restore a UEDGE case. Here, we will look at a few different ways of doing so.

### Restoring a UEDGE case from an input file<a class="anchor" id="inputfile"></a>
UETOOLS can restore UEDGE cases from YAML input files. To do this, you need to have a case set up for usage with an input file. Such an example has been provided as part of the notebook. More information on setting up a YAML input file and UEDGE cases in general, can be found under the relevant sections of the UETOOLS and UEDGE manuals, respectively. 

Now, let's navigate to the folder containing a UEDGE example case and restore the UEDGE case:

In [1]:
# Navigate to the example folder
from os import chdir
chdir('testcase_hires')
# Restore the case
from uetools import Case
c = Case('input.yaml') # Create a test at variable "c" from the YAML input file

UEDGE configuration file /Users/holm10/.uetoolsrc read.
UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_hires/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00



As UETOOLS restores the case from the YAML file, it also checks whether the case is converged. As is evident from the output this example case is converged. The convergence check is done without precoditioning to quickly populate the UEDGE variables based on the case setup and the saved state. The same population of the variables and check for convergence can be done manually at any point, using Case.populate:

In [2]:
c.populate()


*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00



In [3]:
bbb.issfon = 0 # Do not precondition matrix
bbb.ftol = 1e20 # Increase the tolerance to make a successful time-step, so that the UEDGE arrays are updated
bbb.exmain() # Take time-step, calculating and updating all arrays
# Remember to restore the default values
bbb.issfon = 1
bbb.ftol=1e-8 

 Reading grid data from .../testcase_hires/gridue.hdf5.hdf5 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    

 Grid data read successfully:
     file name:   .../testcase_hires/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=      2.272511486499118     nfe=      1


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1


### Restoring a UEDGE case from a save file<a class="anchor" id="savefile"></a>
Alternatively, UETOOL Case objects can also be restored from save-files, without the need of separate input or grid files, as discussed above. In order to create a Case object from a UETOOLS save-file, create a Case with the save-file path as argument:

In [4]:
c = Case('nc20.hdf5')

Restoring case from HDF5 file:
  Rate dirs read from .uedgerc
  Grid read from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_hires/nc20.hdf5
UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_hires/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00



As is evident from the above, restoring from a save-file is equivalent to restoring a save from an input file when using UETOOLS. Therefore, UEDGE cases can be transferred in a single file (barring any atomic and molecular rates, which are used when setting up the case).

### Creating a Case object from memory<a class="anchor" id="memory"></a>
In certain situations, it might be necessary to create a Case object without using either a YAML input or UETOOLS save-file, e.g. when the case only has a Python script \*.py input file available. Under such circumstances, restore the case as one would normally do and, then, create a Case object wihtout any input arguments. This option creates a Case object reading all data from memory, and can be used for converging/analyzing the data:

In [5]:
# <INITIALIZE THE CASE FIRST>
c = Case()
# Ensure the case is converged
c.populate()


*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00



### Static Case objects<a class="anchor" id="static"></a>
Static case objects are Case objects that do not interact with UEDGE but, rather, read data from a file created by UETOOLS, such as a save file. The save file contains a number of commonly-used arrays (see Case Options for more details), which can be accessed using UETOOLS without running UEDGE. Such Case objects are 'read-only', but very useful for accessing results of UEDGE quickly and effortlessly:

In [6]:
c_static = Case('nc20.hdf5', inplace=True)

### Case options<a class="anchor" id="caseoptions"></a>
When creating a case, there are a few keyword options available to the user, which might be useful:
- inplace [=False]: When set to True, a Case is created contining only the UEDGE data in the file defined. This option is icompatible with reading YAML input files, and require specifying a data file of HDF5 type, created by UETOOLS (such as a save file).
- variableyamlfile [=None]: Path to manually defined YAML file, defining additional data to be read/written to save files by UETOOLS. By default, uses the defaultvariables.yaml in UETOOLS/yamls
- verbose [=True]: Setting to False supresses output of Case objects

## Using Case objects to gain information on UEDGE setup and variables<a class="anchor" id="information"></a>
The Case object has a number of useful utilities that can be used to interact with UEDGE. Some of these functions can be imported directly from UEDGE, whereas others cannot.

### Getting an overview of the UEDGE case using Case.about_uedge_setup<a class="anchor" id="aboutsetup"></a>
A utility for getting an overview of the setup of the UEDGE case currently being interacted with is available through the function Case.about_uedge_setup. Presently, the function returns information on the species included in the case in human-readable format. In the future, the plan is to expand the diagnostic capabilites to include information about recycling, pumpin, puffing, and boundary conditions as well.

In [7]:
c.about.uedge_setup()

The UEDGE set-up contains:
  - 2 hydrogenic species:
    - D ions
    - Inertial D atoms
  - 1 impurity species:
    - Charge-state resolved C12
      - Fully force-balance

               0      1      2      3      4      5      6      7  
Ion array : [  D+ ,   D0 ,  C+1 ,  C+2 ,  C+3 ,  C+4 ,  C+5 ,  C+6 ]
Gas array : [  D0 ,   C0 ]


### Searching for variables by name using Case.searchvarname<a class="anchor" id="searchvarname"></a>
Case objects can be used to search for UEDGE variables with names containing a partial string using the function Case.searchvarname. Note that the Case.searchvarname function is case-insensitive.

In [8]:
c.search.varname('Grid')

['rgrid1',
 'igrid',
 'isgriduehdf5',
 'isoldgrid',
 'ngrid',
 'gengrid',
 'manualgrid',
 'gridmorph',
 'GridFileName',
 'rgrid1w',
 'gridx',
 'gridz']

### Searching for variable by description content using Case.search
In case you do not know what a variable might be called, the description of all UEDGE variables can be searched using Case.search. Note that the Case.search function is case-insensitive.

In [9]:
c.search.comments('electron temperature')

['tebbb',
 'res_te',
 'del_te',
 'tinit',
 'te',
 'tel',
 'te0',
 'te_pnc',
 'tezag',
 'tizag',
 'shethp',
 'te0_wdf',
 'tehvt',
 'etemp']

### Getting information on a variable using Case.about<a class="anchor" id="about"></a>
To access information on a specific variable, the Case.about function can be used. Note that Case.about **is** case-sensitive.

In [10]:
c.search.var('GridFileName')

Package:    bbb
Group:      UEint
Attributes: UEint input 
Dimension:  
            (200)
Type:       character(200)
Address:    4569683488
Pyaddress:  4424853584
Unit:       
Comment:
name of Grid file to be read


### Accessing part of a variable description<a class="anchor" id="descriptors"></a>
The different information available through Case.about can be accessed individually using the functions in uetools.UeUtils.Lookup, available through Case.

In [11]:
from uetools.UeUtils.Lookup import Lookup
help(Lookup)
print('te unit is: ', c.search.unit('te'))

Help on class Lookup in module uetools.UeUtils.Lookup:

class Lookup(builtins.object)
 |  Class providing various search functions
 |  
 |  Methods
 |  -------
 |  getpackage(var, verbose=True, **kwargs)
 |      gets package var belongs to as str
 |  getpackobj(var, verbose=True, **kwargs)
 |      gets package var belongs to as Forthon Package obj
 |  var(variable)
 |      print documentation available for var
 |  infostring(variable)
 |      get all info of var as a raw string
 |  aboutdict(variable)
 |      get a parsed dict of into about var
 |  aboutparameter(variable, parameter)
 |      Return "parameter" attribute of var
 |  package(variable)
 |      Return "package" attribute of var
 |  group(variable)
 |      Return "group" attribute of var
 |  attributes(variable)
 |      Return "attributes" attribute of var
 |  dimension(variable)
 |      Return "dimension" attribute of var
 |  type(variable)
 |      Return "type" attribute of var
 |  address(variable)
 |      Return "address

## Managing data using Case objects<a class="anchor" id="managing-data"></a>
Here, managing UEDGE data using Case objects will be shortly discussed, assuming the case is run in a UEDGE session (e.g. inplace=False was used during the creation of the case). 

### Case.populate<a class="anchor" id="populate"></a>
As already mentioned, Case.populate fills two functions:
1. Populates all arrays in UEDGE based on the setup provided and the saved state
2. Assesses whether the case is at steady-state or not 

In [12]:
c.populate() 


*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00



Case.populate executes a series of UEDGE commands that computes all UEDGE arrays in a time-efficient manner without preconditioning the matrix to be solved. However, this results in an increase of the residuals from the level of numerical noise to values closer to unity. In general, cases are coverged if residuals calculated in this manner are <10, but likely not if they are >100. 

The populate function calls the following commands:

In [13]:
bbb.issfon = 0 # Turn off preconditioning to quickly calculate arrays
bbb.ftol = 1e20 # Ensure results are within tolerances to get a succesfull time-step so that all arrays are updated
bbb.exmain() # Take a time-step
# Restore the defaults 
bbb.issfon = 1
bbb.ftol = 1e-8

 Reading grid data from .../testcase_hires/nc20.hdf5.hdf5 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    

 Grid data read successfully:
     file name:   .../testcase_hires/nc20.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=      2.272511486499144     nfe=      1


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1


### Case.get<a class="anchor" id="get"></a>
The Case.get method is used to access UEDGE data or, in the case of inplace=True, data from a file. One advantage of Case.get over directly accessing the UEDGE variables, is that no knowlege of the packages each variable is defined in is necessary, contrary to directly accessing them from UEDGE. However, the usage of Case.get is equivalent to directly accessing variables from UEDGE when run in interactive mode:

In [14]:
print("Interactive ngsp: ", c.get('ngsp'))
print("Static ngsp: ", c_static.get('ngsp'))
# Import UEDGE packages to prompt 
from uedge import *
print("UEDGE-accessed ngsp:", com.ngsp)

Interactive ngsp:  2
Static ngsp:  2
UEDGE-accessed ngsp: 2


### Case.setue<a class="anchor" id="setue"></a>
The Case.setue method can be used to the value of UEDGE variables. Naturally, this function is disabled for static Case objects. Again, the standard UEDGE variables can be used instead of using Case.setue:

In [15]:
c.setue("ngsp", 3)
print("ngsp after being set to 3 by setue:", com.ngsp)
com.ngsp = 2
print("ngsp after being set natively in UEDGE:",com.ngsp)
try:
    c_static.setue("ngsp", 3)
except Exception as e:
    print(repr(e))

ngsp after being set to 3 by setue: 3
ngsp after being set natively in UEDGE: 2
Exception('Cannot set UEDGE values when reading from HDF5 file')


**NOTE:** Changes made to the UEDGE variables are only transferred to the Case object after a time-step has been taken in UEDGE, executing the changes (either using Case.populate or exmain). Consequently, Case.get will return the old values until Case.populate or and exmain has been executed.

### Case.assign<a class="anchor" id="assign"></a>
The Case method uses mutex (mutually exclusive) checks to manage the shared UEDGE memory. This means, only one case interactive object can be active at any single time. Consequently, when a more than one Case object is created in the same Python session is created, only one can be used at a time:

In [16]:
c2 = Case("input.yaml")
try:
    c.get("ngsp")
except Exception as e:
    print(repr(e))

UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_hires/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00

Mutex error! Object run-ID is 3, UEDGE run-ID is 4. Aborting.
Exception("Case doesn't own UEDGE memory")


In order to return control to another Case object, the assign function can be used:

In [17]:
c.assign()
print("ngsp in Case object 'c': ", c.get("ngsp"))

ngsp in Case object 'c':  2


**NOTE:** When switching between cases there may be switches set in one case, but not in the other, which subsequently remain unset. Thus, using several cases in the same session is to be avoided, as they may be order-dependent. If several cases with different input files must be used within the same session, use Case.populate regularly to assure the settings of the case you are working with has not changed:

In [18]:
c.populate()


*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00



## Saving and restoring UEDGE states<a class="anchor" id="save_restore"></a>

Here, the workings of UEDGE when saving and restoring a previous state will be briefly discussed, and some useful functionalities of UETOOLS will be presented. 

The current state of a UEDGE case is defined by a number of "state" variables, separate from their evolving counterparts. Such variables are differentiated by an appended "s", such as "te" vs "tes". Any changes to "te" will not affect the current UEDGE state, whereas changes to "tes" will. When a change to a "state" variable is made, the change will remain in that variable only until a successful time-step, e.g. exmain-evaluation, has been performed. Then, UEDGE will sync the "state" and regular variables.  

In [19]:
c.populate()
print("te[10,10] before: ", bbb.te[10,10])
bbb.te[10,10] = 1e18
c.populate()
print("te[10,10] after: ", bbb.te[10,10])


*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00

te[10,10] before:  1.1729813231213502e-17

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00

te[10,10] after:  1.1729813231213502e-17


As can be seen from the above example, making a change to "te" does not affect the UEDGE solution, as UEDGE reads the current state from "tes". Because Case.populate involves evaluating exmain() (see above), a successful time-step is enforced and "te" is updated to correspond to the original "tes" array, which remained unchanged. This explains the, perhaps somewhat unexpected, behavior observed.

In [20]:
c.populate()
print("te[10,10] before: {:.3e}".format(bbb.te[10,10]))
bbb.tes[10,10] = 2e-17
c.populate()
print("te[10,10] after: {:.3e}".format(bbb.te[10,10]))


*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00

te[10,10] before: 1.173e-17

*** UEDGE arrays populated: WARNING, case NOT converged ***
fnrm without preconditioning: 5.58e+05

te[10,10] after: 2.000e-17


In the above example, "tes" is changed, resulting in a change to the UEDGE state, which now is perturbed from its steady-state, resulting in residuals much above the numerical noise, indicating the case is no longer converged.

### Restoring a UEDGE state using Case.restore_save<a class="anchor" id="restore"></a>
UETOOLS has internal capabilites for finding, reading and restoring the appropriate data from a file:

In [21]:
print("te[10,10] before: {:.3e}".format(bbb.te[10,10]))
c.restore_save('nc20.hdf5')
print("te[10,10] after: {:.3e}".format(bbb.te[10,10]))

te[10,10] before: 2.000e-17
UETOOLS-style save successfully restored from nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 2.27e+00

te[10,10] after: 1.173e-17


As can be seen from above, restoring the saved state returns the residuals as well as "te" to their initial values.

### Saving a UEDGE state using Case.save<a class="anchor" id="save"></a>
Whenever a UEDGE case has been reconverged (as explained in Ch. 4), the current UEDGE state should be saved for later recovery:

In [22]:
print("testsave.hdf5 before:")
!ls -lrt testsave.hdf5

# Save state
c.save('testsave.hdf5')

print("testsave.hdf5 after:")
!ls -lrt testsave.hdf5

testsave.hdf5 before:
-rw-r--r--  1 holm10  27647  1636464 Aug  1 15:35 testsave.hdf5
testsave.hdf5 after:
-rw-r--r--  1 holm10  27647  1636464 Aug  1 15:35 testsave.hdf5
